In [1]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float, ForeignKey
from sqlalchemy.orm import sessionmaker, relationship

In [2]:
engine = create_engine("sqlite:///yummydata.sqlite")

In [3]:
Base = declarative_base()

In [4]:
# Create ORM Class for Zip Code Demographic Data
class ZipDemo(Base):
    
    __tablename__ = 'zip_demographics'

    id = Column(Integer, primary_key=True)
    Zip = Column(Integer)
    Population = Column(Integer)
    Density = Column(Float)
    AverageIncome = Column(Float)
    City = Column(Text)
    State = Column(Text)
    ZipLatitude = Column(Float)
    ZipLongitude = Column(Float)
    
#     def __repr__(self):
#         return f"id={self.id}, name={self.station}"

In [5]:
# NOTE - CHANGED COLUMN NAMES WITH PARENTHESES (NEED TO DO THIS ON THE API CODE)
class RestaurantSearch(Base):
    
    __tablename__ = 'restaurant_search'

    id = Column(Integer, primary_key=True)
    Address = Column(Text)
    Google_ID = Column(Text)
    Latitude = Column(Float)
    Longitude = Column(Float)
    Name = Column(Text)
    Price_Level = Column(Integer)
    Rating = Column(Float)
    Zip = Column(Integer)

In [6]:
class Input(Base):
    
    __tablename__ = 'user_input'

    id = Column(Integer, primary_key=True)
    City = Column(Text)
    State = Column(Text)
    Cuisine = Column(Text)

In [7]:
Base.metadata.create_all(engine)

In [8]:
# Create a Functin to Populate Tables
def populate_table(engine, table, csvfile):
    # connect to the database
    conn = engine.connect()
    
    # Load the CSV file into a pandas dataframe 
    df_of_data_to_insert = pd.read_csv(csvfile)
    
    # Orient='records' creates a list of data to write
    data = df_of_data_to_insert.to_dict(orient='records')

    # Optional: Delete all rows in the table 
    conn.execute(table.delete())

    # Insert the dataframe into the database in one bulk insert
    conn.execute(table.insert(), data)

In [9]:
# Populate Tables
populate_table(engine, ZipDemo.__table__, 'zip_demographicchineseATX.csv')
populate_table(engine, RestaurantSearch.__table__, 'chineseATX.csv')